# Detecting change in Malaysian forestry <img align="right" src="../resources/csiro_easi_logo.png">

This notebook has been adapted from [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Real_world_examples/Chlorophyll_monitoring.ipynb)
* **Products used:** 
[s2_l2a](http://explorer.asia.easi-eo.solutions/products/s2_l2a)

## Background
Methods for detecting meaningful and significant change in forests are important for those who manage and monitor large areas of forest.

On-the-ground monitoring can be expensive and time-consuming, especially when forests are in difficult-to-navigate terrain.
Aerial photography and LiDAR can provide detailed information about forests, but are often extremely expensive to acquire, even over small areas.

### Sentinel-2 use case
Satellite imagery from the [EU Copernicus Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) is freely available.
Its 10 metre resolution makes it perfect for monitoring fine changes over very large areas of land.
The archive of Sentinel-2 data over Southeast Asia stretches back to 2017, meaning that there is a good amount of data for change detection, allowing us to average out or focus on seasonal changes.

## Description
In this example, we measure the presence of vegetation from Sentinel-2 imagery and apply a hypothesis test to identify areas of significant change (along with the direction of the change).
The worked example takes users through the code required to do the following:

1. Load cloud-free Sentinel-2 images for an area of interest
2. Compute an index to highlight presence of vegetation (NDVI)
3. Apply a statistical hypothesis test to find areas of significant change
4. Visualise the statistically significant areas.

***

## Getting started

**To run this analysis**, run all the cells in the notebook, starting with the "Load packages" cell.

**After finishing the analysis**, return to the "Analysis parameters" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.

### Load packages
Load key Python packages and any supporting functions for the analysis.

In [ ]:
%matplotlib inline

import datacube
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from datacube.utils import masking
from datacube.utils.cog import write_cog
from datacube.utils.geometry import CRS

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.plotting import rgb, display_map

### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [ ]:
dc = datacube.Datacube(app="Change_detection")

### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
There is also a parameter to define how the data is split in time; the split yields two non-overlapping samples, which is a requirement of the hypothesis test we want to run (more detail below).
The parameters are:

* `latitude`: The latitude range to analyse (e.g. `(1.2449, 1.195)`).
For reasonable loading times, make sure the range spans less than ~0.1 degrees.
* `longitude`: The longitude range to analyse (e.g. `(110.5658, 110.6158)`).
For reasonable loading times, make sure the range spans less than ~0.1 degrees.
* `time`: The date range to analyse (e.g. `('2018-01-01', '2020-12-31')`).
Note that Sentinel-2 data is not available in Southeast Asia prior to 2017.
For reasonable results, the range should span at least two years to prevent detecting seasonal changes.
* `time_baseline`: The date at which to split the total sample into two non-overlapping samples (e.g. `'2019-06-01'`).
For reasonable results, pick a date that is about halfway between the start and end dates specified in `time`.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example covers a commercial plantation located near Serain, Malaysia.

**To run the notebook for a different area**, make sure Sentinel-2 data is available for the chosen area using the [EASI CSIRO Asia Explorer](http://explorer.asia.easi-eo.solutions/products/s2_l2a).
Use the drop-down menu to check Sentinel-2a and Sentinel-2b (`s2_l2a`).

In [ ]:
# Define the area of interest
longitude = (110.5658, 110.6158)
latitude = (1.2449, 1.195)

# Set the range of dates for the complete sample
time = ('2018-01-01', '2020-12-31')

# Set the date to separate the data into two samples for comparison
time_baseline = '2019-06-01'

## View the selected location
The next cell will display the selected area on an interactive map.
The red border represents the area of interest of the study.
Zoom in and out to get a better understanding of the area of interest.
Clicking anywhere on the map will reveal the latitude and longitude coordinates of the clicked point.

In [ ]:
display_map(x=longitude, y=latitude)

## Load and view Sentinel-2 data
The first step in the analysis is to load Sentinel-2 data for the specified area of interest and time range.

The loading process makes use of dask, which will return a lazy-loaded data set. This means that no values will be loaded until abosultely required. As such, the next cell will run quickly, but you won't yet be able to visualise the values.

The load is complete when the cell status goes from `[*]` to `[number]`.

In [ ]:
# Choose products to load
# Here, the Sentinel-2 product is chosen
product = ['s2_l2a']

# Specify the parameters to pass to the load query
query = {
    "x": longitude,
    "y": latitude,
    "time": time,
    "measurements": [
        "red",  # Red band
        "green",  # Green band
        "blue",  # Blue band
        "nir_1",  # Near-infrared band
        "mask",  # Cloud masking band
    ],
    "output_crs": "EPSG:3577",
    "resolution": (-20, 20),
    "dask_chunks": {"time":1, "x":350, "y":300}
}

# Load the data
ds_s2 = dc.load(product=product, **query)

**Once the load is complete**, examine the data by printing it in the next cell.
The `Dimensions` argument revels the number of time steps in the data set, as well as the number of pixels in the x (longitude) and y (latitude) dimensions.

In [ ]:
ds_s2

### Masking

Clouds, cloud shadows, and other invalid pixels need to be removed before conducting the analysis.

The first step in masking is to identify the information in the masking band. This can be done using the `masking.describe_variable_flags)` function:

In [ ]:
masking.describe_variable_flags(ds_s2.mask)

For Sentinel-2 in the Asia EASI Hub, the masking information is stored in the **values** column of the **qa** row in this dataframe. To get a better look at the types of pixel classifications this mask has, we can display the values column:

In [ ]:
masking.describe_variable_flags(ds_s2.mask).loc["qa", "values"]

We can now construct a mask of valid pixel classes. In this case, we say that `vegetation`, `bare soils`, `water`, or `snow or ice` are valid, cloud-free pixels:

In [ ]:
# Multiple flags are combined as logial OR using the | symbol
cloud_free_mask = (
    masking.make_mask(ds_s2.mask, qa="vegetation") | 
    masking.make_mask(ds_s2.mask, qa="bare soils") |
    masking.make_mask(ds_s2.mask, qa="water") |
    masking.make_mask(ds_s2.mask, qa="snow or ice")
)

In [ ]:
cloud_free_mask

Now that we have a mask that contains the location of valid pixels, we can count the number of valid pixels and divide by the total number of pixels to identify the proportion of valid data in each observation.

In [ ]:
# Calculate proportion of good pixels
valid_pixel_proportion = cloud_free_mask.sum(dim=("x", "y"))/(cloud_free_mask.shape[1] * cloud_free_mask.shape[2])

valid_threshold = 0.7
observations_to_keep = (valid_pixel_proportion >= valid_threshold)

The final step is to mask the data, then only keep the observations where more than 70% of the pixels are valid. 

**Please be patient**. The data may take a few minutes to load and progress will be indicated by text output. The load is complete when the cell status goes from `[*]` to `[number]`.

In [ ]:
# Mask the data
ds_s2_valid = ds_s2.where(cloud_free_mask)

# only keep observations above the good pixel proportion threshold
# The .compute() step means the values will be loaded into memory. This step may take some time
ds_s2_keep = ds_s2_valid.sel(time=observations_to_keep).compute()

Once the load is complete, examine the data by printing it in the next cell. The `[Dimensions]` argument reveals the number of time steps in the dataset. 

In [ ]:
# view the loaded data
ds_s2_keep

### Plot example timestep in true colour

To visualise the data, use the pre-loaded `rgb` utility function to plot a true colour image for a given time-step.
White spots in the images are where clouds have been masked out.

The settings below will display images for two time steps, one in mid August 2018, one in early September 2019.
Can you spot any areas of change?

Feel free to experiement with the values for the `initial_timestep` and `final_timestep` variables; re-run the cell to plot the images for the new timesteps.
The values for the timesteps can be `0` to one fewer than the number of time steps loaded in the data set.
The number of time steps is the same as the total number of observations listed as the output of the cell used to load the data.

> **Note:** If the location and time are changed, you may need to change the `intial_timestep` and `final_timestep` parameters to view images at similar times of year.

In [ ]:
# Set the timesteps to visualise
initial_timestep = 8
final_timestep = 19

# Generate RGB plots at each timestep
rgb(ds_s2_keep, bands=["red", "green", "blue"], index=[initial_timestep, final_timestep])

## Compute band indices
This study measures vegetation through the normalised difference vegetation index (NDVI), which can be calculated using the predefined `calculate_indices` utility function.
This index uses the ratio of the red and near-infrared (NIR) bands to identify live green vegetation. 
The formula is

$$
\begin{aligned}
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}.
\end{aligned}
$$

When interpreting this index, high values indicate vegetation, and low values indicate soil or water.

In [ ]:
# Calculate NDVI and add it to the loaded dataset
ds_s2_keep["NDVI"] = (ds_s2_keep.nir_1 - ds_s2_keep.red)/(ds_s2_keep.nir_1 + ds_s2_keep.red)

The plots below show the NDVI values for the two selected timesteps used to make the true-colour images above.
Use the plots to visually confirm whether NDVI is a suitable index for change detection.

In [ ]:
# Plot the NDVI values for pixels classified as water for the two dates.
ds_s2_keep.NDVI.isel(time=[initial_timestep, final_timestep]).plot.imshow(
    'x', 'y', col='time', cmap='RdYlGn', vmin=0, vmax=1, figsize=(18, 6))

plt.show()

## Perform hypothesis test
While it is possible to visually detect change between the `2018-08-20` and `2019-09-04` timesteps, it is important to consider how to rigorously check for both positive change in the NDVI (afforestation) and negative change in the NDVI (deforestation).

This can be done through hypothesis testing.
In this case, 

$$
\begin{aligned}
\text{null hypothesis } (H_0) &: \text{no change occurred,} \\
\text{alternative hypothesis } (H_1) &: \text{some change occurred.}
\end{aligned}
$$

The hypothesis test will indicate where there is evidence for rejecting the null hypothesis.
From this, we may identify areas of signficant change, according to a given significance level (covered in more detail below).

### Make samples

To perform the test, the total sample will be split in two: a `baseline` sample and a `postbaseline` sample, which respectively contain the data before and after the `time_baseline` date.
Then, we can test for a difference in the average NDVI between the samples for each pixel in the sample.

The samples are made by selecting the `NDVI` band from the dataset and filtering it based on whether it was observed before or after the `time_baseline` date.
The number of observations in each sample will be printed.
If one sample is much larger than the other, consider changing the `time_baseline` parameter in the "Analysis parameters" cell, and then re-run this cell.
Coordinates are recorded for later use.

In [ ]:
# Make samples
baseline_sample = ds_s2_keep.NDVI.sel(time=ds_s2_keep['time']<=np.datetime64(time_baseline))
print(f"Number of observations in baseline sample: {len(baseline_sample.time)}")

postbaseline_sample = ds_s2_keep.NDVI.sel(time=ds_s2_keep['time']>np.datetime64(time_baseline))
print(f"Number of observations in postbaseline sample: {len(postbaseline_sample.time)}")

# Record coodrinates for reconstructing xarray objects
sample_lat_coords = ds_s2.coords['y']
sample_lon_coords = ds_s2.coords['x']

### Test for change
To look for evidence that the average NDVI has changed between the two samples (either positively or negatively), we use Welch's t-test.
This is used to test the hypothesis that two populations have equal averages.
In this case, the populations are the area of interest before and after the `time_baseline` date, and the average being tested is the average NDVI.
Welch's *t*-test is used (as opposed to Student's *t*-test) because the two samples in the study may not necessarily have equal variances. 

The test is run using the Scipy package's statistcs library, which provides the `ttest_ind` function for running *t*-tests. 
Setting `equal_var=False` means that the function will run Welch's *t*-test.
The function returns the *t*-statistic and *p*-value for each pixel after testing the difference in the average NDVI.
These are stored as `t_stat` and `p_val` inside the `t_test` dataset for use in the next section.

In [ ]:
# Perform the t-test on the postbaseline and baseline samples
tstat, p_tstat = stats.ttest_ind(
    postbaseline_sample.values,
    baseline_sample.values,
    equal_var=False,
    nan_policy='omit',
)

# Convert results to an xarray for further analysis
t_test = xr.Dataset(
    {
        't_stat': (['y', 'x'], tstat),
        'p_val': (['y', 'x'], p_tstat)
    },
    coords={
        'x': (['x'], sample_lon_coords.values),
        'y': (['y'], sample_lat_coords.values)
    }, 
    attrs={
        'crs': 'EPSG:3577',
    })

print(t_test)

## Visualise change
From the test, we're interested in two conditions: whether the change is significant (rejection of the null hypothesis) and whether the change was positive (afforestation) or negative (deforestation).

The null hypothesis can be rejected if the $p$-value (`p_val`) is less than the chosen significance level, which is set as `sig_level = 0.05` for this analysis.
If the null hypothesis is rejected, the pixel will be classified as having undergone significant change.

The direction of the change can be inferred from the difference in the average NDVI of each sample, which is calculated as $$\text{diff mean} = \text{mean(post baseline)} - \text{mean(baseline)}.$$
This means that:

* if the average NDVI for a given pixel is **higher** in the `post baseline` sample compared to the `baseline` sample, then `diff_mean` for that pixel will be **positive**.
* if the average NDVI for a given pixel is **lower** in the `post baseline` sample compared to the `baseline` sample, then `diff_mean` for that pixel will be **negative**.

Run the cell below to first plot the difference in the mean between the two samples, then plot only the differences that were marked as signficant. 
**Positive change is shown in blue and negative change is shown in red.**

In [ ]:
# Set the significance level
sig_level = 0.05

# Plot any difference in the mean
diff_mean = postbaseline_sample.mean(dim=['time']) - baseline_sample.mean(dim=['time'])

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
diff_mean.plot(cmap='RdBu')
ax.set_title('Any difference in the mean')
plt.show()

# Plot any difference in the mean classified as significant
sig_diff_mean = postbaseline_sample.mean(dim=['time']).where(t_test.p_val < sig_level) - baseline_sample.mean(dim=['time']).where(t_test.p_val < sig_level)

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
sig_diff_mean.plot(cmap='RdBu')
ax.set_title('Statistically significant difference in the mean')
plt.show()

## Drawing conclusions
Here are some questions to think about:

* What has happened in the forest over the time covered by the dataset?
* Were there any statistically significant changes that the test found that you didn't see in the true-colour images? 
* What kind of activities/events might explain the significant changes?
* What kind of activities/events might explain non-significant changes?
* What other information might you need to draw conclusions about the cause of the statistically significant changes?

## Export the data
To explore the data further in a desktop GIS program, the data can be output as a GeoTIFF. 
The `diff_mean` product will be saved as "ttest_diff_mean.tif", and the `sig_diff_mean` product will be saved as "ttest_sig_diff_mean.tif".
These files can be downloaded from the file explorer to the left of this window ([see these instructions](https://jupyterlab.readthedocs.io/en/stable/user/files.html#uploading-and-downloading)).

In [ ]:
write_cog(geo_im=diff_mean, fname="ttest_diff_mean.tif", overwrite=True)
write_cog(geo_im=sig_diff_mean, fname="ttest_sig_diff_mean.tif", overwrite=True)

## Next steps

When you are done, return to the "Analysis parameters" section, modify some values (e.g. `latitude`, `longitude`, `time` or `time_baseline`) and re-run the analysis.
You can use the interactive map in the "View the selected location" section to find new central latitude and longitude values by panning and zooming, and then clicking on the area you wish to extract location values for.
You can also use Google maps to search for a location you know, then return the latitude and longitude values by clicking the map.

If you're going to change the location, you'll need to make sure Sentinel-2 data is available for the new location, which you can check at the [EASI CSIRO Asia Explorer](http://explorer.asia.easi-eo.solutions/products/s2_l2a).
Use the drop-down menu to check Sentinel-2a and Sentinel-2b (`s2_l2a`).